In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import math

In [28]:
path = 'drive/MyDrive/LDS/DataSet/tennis_no_missing.csv'

In [29]:
df = pd.read_csv(path)

In [31]:
df['tourney_to_datetime'] = pd.to_datetime(df['tourney_date'], format='%Y%M%d')

In [32]:
df['year'] = df['tourney_to_datetime'].dt.year

In [33]:
col = ['winner_id','loser_id','loser_rank', 'loser_rank_points','winner_rank','winner_rank_points', 'year']

In [34]:
df[col].isna().sum()

winner_id                 0
loser_id                  0
loser_rank            35206
loser_rank_points     35222
winner_rank           19344
winner_rank_points    19360
year                      0
dtype: int64

In [35]:
df[col][ df['loser_rank_points'].isna() & df['winner_rank_points'].isna()]

,winner_id,loser_id,loser_rank,loser_rank_points,winner_rank,winner_rank_points,year
112,104431,101339,NaN,NaN,NaN,NaN,2016
369,132663,103417,NaN,NaN,NaN,NaN,2016
515,104604,103600,NaN,NaN,NaN,NaN,2016
606,126327,103612,NaN,NaN,NaN,NaN,2017
609,202320,103612,NaN,NaN,NaN,NaN,2017
...,...,...,...,...,...,...,...
185572,214003,245146,NaN,NaN,NaN,NaN,2018
185575,214257,245146,NaN,NaN,NaN,NaN,2019
185577,216262,245146,NaN,NaN,NaN,NaN,2016
185592,216146,245148,NaN,NaN,NaN,NaN,2017


#### Winner ranks e Loser ranks MV imputation

In [36]:
# tutti i winner_id che hanno almeno un winner_rank mancante
winner_miss_rank = df['winner_id'].loc[df['winner_rank'].isna()].unique().tolist()

In [37]:
len(winner_miss_rank)

4008

imputazione di winner_rank con il winner_rank NOT NULL dello stesso winner_id che ha giocato nello stesso anno (il rank di un giocatore, infatti, viene aggiornato ogni 52 settimana, quindi questo metodo può essere una buona stima)

In [38]:
# per ogni giocatore vincente che ha almeno un winner_rank mancante vorremmo provare a trovare un altro winner_rank
# per quel determinato giocatore in quel determinato anno 

rk = df[['winner_rank','winner_id','year']][( df['winner_id'].isin(winner_miss_rank) ) ]
# rk

In [39]:
# tutti i record con winner_rank NOT NULL dei winner_id che hanno almeno un winner_rank mancante 
rk_winner_notnull = rk[rk['winner_rank'].notnull()]
# rk_winner_notnull

In [40]:
# tutti i record con winner_rank NULL dei winner_id che hanno almeno un winner_rank mancante 
rk_winner_isna = rk[rk['winner_rank'].isna()]
# rk_winner_isna

In [41]:
# mergiamo i due DATAFRAME su winner_id e year. 
df_merge_winner_rank = pd.merge(rk_winner_isna, rk_winner_notnull, on=['winner_id', 'year'])
df_merge_winner_rank

,winner_rank_x,winner_id,year,winner_rank_y
0,NaN,103938,2016,1223.0
1,NaN,103938,2016,1223.0
2,NaN,123983,2019,348.0
3,NaN,123983,2019,348.0
4,NaN,104604,2016,933.0
...,...,...,...,...
58096,NaN,207989,2019,566.0
58097,NaN,207989,2019,597.0
58098,NaN,207989,2019,566.0
58099,NaN,213850,2016,1146.0


In [42]:
# creiamo un dizionario con key -->('winner_id', 'year') e value -> 'winner_rank_y'
dict_winner_rank = {}
for index, row in df_merge_winner_rank.iterrows():
    dict_winner_rank[(row['winner_id'], row['year'])] = row['winner_rank_y']

In [43]:
# imputiamo i missing values di winner_rank derivandoli dal dizionario appena creato
for k,v in dict_winner_rank.items():
  df.loc[ (df['winner_rank'].isna()) & (df['winner_id'] == k[0]) & (df['year'] == k[1] ) , 'winner_rank'] = v

In [44]:
df[col].isna().sum()

winner_id                 0
loser_id                  0
loser_rank            35206
loser_rank_points     35222
winner_rank           11960
winner_rank_points    19360
year                      0
dtype: int64

In [45]:
# proviamo a vedere se riusciamo a derivarli dai loser_id, attuando lo stesso meccanismo. 

In [46]:
# per ogni giocatore vincente che ha almeno un winner_rank mancante vorremmo provare a trovare un altro winner_rank
# per quel determinato giocatore in quel determinato anno 

rk = df[['loser_rank','loser_id','year']][( df['loser_id'].isin(winner_miss_rank) ) ]
# rk

In [47]:
# tutti i record con loser_rank NOT NULL dei winner_id che hanno almeno un winner_rank mancante 
rk_loser_notnull = rk[rk['loser_rank'].notnull()]
rk_loser_notnull

,loser_rank,loser_id,year
126,848.0,102864,2016
127,984.0,102864,2016
128,852.0,102864,2016
131,973.0,102864,2016
200,785.0,103175,2016
...,...,...,...
185573,1096.0,245146,2016
185576,1137.0,245146,2016
185578,1145.0,245146,2018
185579,1228.0,245147,2017


In [48]:
# creiamo un dizionario con key -->('loser_id', 'year') e value -> 'loser_rank'
dict_loser_rank = {}
for index, row in rk_loser_notnull.iterrows():
    dict_loser_rank[(row['loser_id'], row['year'])] = row['loser_rank']

In [49]:
# imputiamo i missing values di winner_rank derivandoli dal dizionario appena creato
for k,v in dict_loser_rank.items():
  df.loc[ (df['winner_rank'].isna()) & (df['winner_id'] == k[0]) & (df['year'] == k[1] ) , 'winner_rank'] = v

In [50]:
df[col].isna().sum()

winner_id                 0
loser_id                  0
loser_rank            35206
loser_rank_points     35222
winner_rank           10955
winner_rank_points    19360
year                      0
dtype: int64

Facciamo La stessa cosa per i loser_rank che mancano

In [51]:
# tutti i loser_id che hanno almeno un loser_rank mancante
loser_miss_rank = df['loser_id'].loc[df['loser_rank'].isna()].unique().tolist()

In [52]:
len(loser_miss_rank)

7677

In [53]:
# per ogni giocatore perdente che ha almeno un loser_rank mancante vorremmo provare a trovare un altro loser_rank
# per quel determinato giocatore in quel determinato anno 

rk = df[['loser_rank','loser_id','year']][( df['loser_id'].isin(loser_miss_rank) ) ]
# rk

In [54]:
rk_loser_notnull = rk[rk['loser_rank'].notnull()]

In [55]:
rk_loser_isna = rk[rk['loser_rank'].isna()]

In [56]:
# creiamo un dizionario con key -->('loser_id', 'year') e value -> 'loser_rank'
dict_loser_rank = {}
for index, row in rk_loser_notnull.iterrows():
    dict_loser_rank[(row['loser_id'], row['year'])] = row['loser_rank']

In [57]:
# imputiamo i missing values di winner_rank derivandoli dal dizionario appena creato
for k,v in dict_loser_rank.items():
  df.loc[ (df['loser_rank'].isna()) & (df['loser_id'] == k[0]) & (df['year'] == k[1] ) , 'loser_rank'] = v

In [58]:
df[col].isna().sum()

winner_id                 0
loser_id                  0
loser_rank            26709
loser_rank_points     35222
winner_rank           10955
winner_rank_points    19360
year                      0
dtype: int64

In [59]:
# proviamo a vedere se riusciamo a derivarli dai winner_id, attuando lo stesso meccanismo. 

In [60]:
rk = df[['winner_rank','winner_id','year']][( df['winner_id'].isin(loser_miss_rank) ) ]

In [61]:
rk_winner_notnull = rk[rk['winner_rank'].notnull()]

In [62]:
# creiamo un dizionario con key -->('winner_id', 'year') e value -> 'winner_rank'
dict_winner_rank = {}
for index, row in rk_winner_notnull.iterrows():
    dict_winner_rank[(row['winner_id'], row['year'])] = row['winner_rank']

In [63]:
# imputiamo i missing values di loser_rank derivandoli dal dizionario appena creato
for k,v in dict_winner_rank.items():
  df.loc[ (df['loser_rank'].isna()) & (df['loser_id'] == k[0]) & (df['year'] == k[1] ) , 'loser_rank'] = v

In [65]:
#questo nome supera i 50 caratteri fissati per la colonna nel db
df.loc[df['tourney_name'] == 'Southsea 16 Main Draw Friday Final No Qualifying $10000K', 'tourney_name'] = 'Southsea 16 Main Draw Friday Final No Qualifying'

In [66]:
df.drop(columns = ['Unnamed: 0', 'gender', 'year', 'tourney_to_datetime'], inplace= True)

In [69]:
num_col = ['winner_ht', 'winner_age', 'loser_ht', 'loser_age','winner_rank', 'winner_rank_points',
       'loser_rank', 'loser_rank_points']

In [70]:
df[num_col] = df[num_col].fillna(-1)

In [71]:
df.to_csv('drive/MyDrive/LDS/DataSet/tennis_FINALE.csv',index=False)